# Pfadplanung

Bisher haben wir einen "brainless" Roboter erschaffen, der nur auf unmittelbare Sensormessungen reagieren kann. Wenn jedoch der Zustand des Roboter und der Arbeitsraum bekannt sind, können wir den besten Pfad des Roboters im Voraus planen. Dazu zerlegen wir den Arbeitsraum zunächst in definierte Punkte, die der Roboter ansteuern kann. Eine einfache Methode, ist es, ein Raster über den gesamten Arbeitsraum zu legen. Im folgenden Beispiel werden wir eine Welt mir einer Größe von 50 x 50 erstellen. Wir werden ein Raster mit einer AUflösung von 1 nutzen, das heißt die Welt hat 2500 Rasterpunkte.

## Der freie Arbeitsraum

Wir wollen den freien Arbeitsraum für unsere Welt darstellen. Dazu erstellen wir zunächst die Welt in der Größe 50 x 50, fügen die Hindernisse hinzu und definieren Start und Endpunkt. 

In [ ]:
#imports

import sys
sys.path.append('mobilerobot')
import numpy as np
from world import SpaceWorld
from obstacle import Obstacle

In [ ]:
# initialize world
o1 = Obstacle(10, 20, 30, 5)
o2 = Obstacle(10, 20, 5, 15)
o3 = Obstacle(35, 20, 5, 15)

start = (25, 40)
goal = (25, 10)

width = 50
height = 50

BLOCKED = 0
FREE = 1
GOAL = 2
START = 3
VISITED = 4
PATH = 5

def showWorld(field):
    field[start] = Space.START.value
    field[goal] = Space.GOAL.value

    world = SpaceWorld(width, height, obstacles=[o1,o2,o3])
    world.setBackground(field)
    world.showScene()

**Aufgabe**: Erstelle zwei verschachtelte Schleifen. Die erste iteriert über die Breite (``i in range(width)``) und die zweite über die Höhe (``j in range(width)``). Das ``world``-Modul stellt eine Funktion bereit, mit der wir bestimmen können, ob eine bestimmte Position erreichbar ist. Wenn diese Funktion (``world.freeSpace(x=i,y=j,radius=3)``) ``True`` zurückgibt, setze den Wert in ``fields[i][j]`` auf ``FREE``.

In [ ]:
def initializeField():
    field = np.zeros((width,height))
    # ------ add your code here ---------


    # -----------------------------------
    return field

showWorld(initializeField())

## Der Waldbrand-Algorithmus

Der Waldbrand-Algorithmus ist ein einfacher Pfadplanungsalgorithmus, der Schritt für Schritt seine Umwelt erforscht. Beginnend beim Startrasterpunkt erkunden wir die Rasterpunkte oberhalb, unterhalb, rechts und links von diesem Rasterpunkt. Wenn sich die Rasterpunkte im freien Raum befinden erkunden wir in der nächsten Iteration deren direkten Nachbarn, bis wir entweder keine freien Rasterpunkte mehr finden, oder den Zielpunkt gefunden haben.

In der Liste ``exploring`` speichern wir alle Rasterpunkte ab, die wir erkunden wollen. Zunächst ist dies nur der Startrasterpunkt ``start``. Dessen Status setzten wir auf ``VISITED``. Solange diese Liste nicht leer ist erkunden wir die direkten Nachbarn des ersten Rasterpunktes in der Liste.

**Aufgabe:** Ergänze die ``while`` Schleife, sodass in jeder Iteration die direkten Nachbarn (oben, unten, rechts, links) des aktuellen Rasterpunktes ``(x,y)`` erkundet werden. Füge dazu die Rasterpunkte der ``exploring`` Liste mit ``append`` hinzu, wenn der Rasterpunkt den Status ``FREE`` hat. Setzte den Status anschließend auf ``VISITED``, damit er nicht mehrfach erkundet wird.

In [ ]:
def findGoal():
    field = initializeField()

    exploring = [start]
    field[start] = VISITED

    # Begrenzung der Iterationen auf 2000, um Endlosschleife zu vermeiden
    iterations = 0
    timeout = 2000

    while len(exploring) > 0 and iterations < timeout:
        # erster Rasterpunkt in der Liste
        (x, y) = exploring.pop(0)
        # Ziel wurde gefunden
        if (x,y) == goal:
            break
        # ----- add your code here -----------
        


        # ------------------------------------
        iterations += 1
    
    return field


showWorld(findGoal())

Auch wenn wir jetzt das Ziel gefunden haben, wissen wir noch nicht, was der kürzeste Weg vom Start- zum Zielpunkt ist. Dazu müssen wir uns bei jeder Iteration speichern, welcher Rasterpunkt der unmittelbare Vorgänger der Zelle war. Dazu nutzen wir das Array ``parents``.

**Aufgabe:** Kopiere deinen Code aus der oberen Zelle und ergänze ihn so, dass beim Erkunden einer Zelle zusätzlich der Vorgängerrasterpunkt in parents gepeichert wird. Befinden wir uns beispielsweise an der Position (5, 17) und erkunden die Zelle (6,17), rufe ``parents[6][17] = (5,17)`` auf.

In [ ]:
def findPath():
    field = initializeField()

    exploring = [start]
    field[start] = VISITED

    # Array zum Speichern des Vorgängers für jeden Rasterpunkt
    parents = [[None for i in range(width)] for j in range(height)]

    # Begrenzung der Iterationen auf 2000, um Endlosschleife zu vermeiden
    iterations = 0
    timeout = 2000

    while len(exploring) > 0 and iterations < timeout:
        # erster Rasterpunkt in der Liste
        (x, y) = exploring.pop(0)
        # Ziel wurde gefunden
        if (x,y) == goal:
            break
        # ----- add your code here -----------
        


        # ------------------------------------
        iterations += 1
    
    return field, parents

field, parents = findPath()

def getParent(p):
  (x,y) = p
  return parents[x][y]

current = goal
while current is not None:
  (x, y) = current
  field[x][y] = Space.PATH.value
  current = getParent(current)

showWorld(findGoal())